## Claims Extraction
#### Using discourse and claim detection paper here: https://arxiv.org/abs/1907.00962
#### github link of the model by author:  https://github.com/titipata/detecting-scientific-claim

In [1]:
import pandas as pd
import os
os.chdir("detecting-scientific-claim")
import preprocess as pp

import spacy
from tqdm import tqdm

nlp = spacy.load('en_core_web_sm')

import os
import sys
import json
from itertools import chain
import torch
from torch.nn import ModuleList, Linear
import torch.nn.functional as F
import numpy as np
import pandas as pd
from nltk import word_tokenize, sent_tokenize

from lxml import etree, html
import urllib

import flask
from flask import Flask, request
from gevent.pywsgi import WSGIServer

from allennlp.models.archival import load_archive
from allennlp.predictors import Predictor
from allennlp.common.file_utils import cached_path
from allennlp.common.util import JsonDict
from allennlp.data import Instance
from allennlp.modules import Seq2VecEncoder, TimeDistributed, TextFieldEmbedder, ConditionalRandomField, FeedForward

from discourse import read_json
from discourse.dataset_readers import ClaimAnnotationReaderJSON, CrfPubmedRCTReader
from discourse.predictors import DiscourseClassifierPredictor


In [2]:
import gc
gc.collect()

55

In [3]:
class ClaimCrfPredictor(Predictor):
    """"
    Predictor wrapper for the AcademicPaperClassifier
    """
    def _json_to_instance(self, json_dict: JsonDict) -> Instance:
        print(json_dict)
        sentences = json_dict['sentences']
        instance = self._dataset_reader.text_to_instance(sents=sentences)
        return instance


!wget https://s3-us-west-2.amazonaws.com/pubmed-rct/model_crf.tar.gz ## the  CRF model weight

In [4]:
archive = load_archive("./model_crf.tar.gz") ## available at github
predictor = Predictor.from_archive(archive, 'discourse_crf_predictor')
archive_ = load_archive("./model_crf.tar.gz")
discourse_predictor = Predictor.from_archive(archive_, 'discourse_crf_predictor')
gc.collect()

/home/harsh/anaconda3/envs/Coronawhy/lib/python3.8/site-packages/torch/nn/modules/rnn.py:47: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


31

In [21]:
WEIGHT_PATH="https://s3-us-west-2.amazonaws.com/pubmed-rct/model_crf_tf.th"
model = predictor._model
for param in list(model.parameters()):
    param.requires_grad = False ## not train weights
EMBEDDING_DIM = 300
num_classes, constraints, include_start_end_transitions = 2, None, False
model.crf = ConditionalRandomField(num_classes, constraints, 
                                   include_start_end_transitions=include_start_end_transitions)
model.label_projection_layer = TimeDistributed(Linear(2 * EMBEDDING_DIM, num_classes))
model.load_state_dict(torch.load(cached_path(WEIGHT_PATH), map_location='cpu'))
reader = CrfPubmedRCTReader()
claim_predictor = ClaimCrfPredictor(model, dataset_reader=reader)

In [29]:
text_input="The present world is facing a devastating reality as drug abuse prevails in every corner of a society. The progress of a country is obstructed due to the excessive practice of taking drugs by the young generation. Like other countries, Bangladesh is also facing this dreadful situation. The multiple use of drug substances leads an individual to a sorrowful destination and for this reason, the natural behavior of human mind is disrupted. An addicted individual may regain his normal life by proper monitoring and treatment. The objective of this study is to analyze a mathematical model on the dynamics of drug abuse in the perspective of Bangladesh and reduce the harmful consequences with effective control policies using the idea of optimal control theory. The model has been solved analytically introducing a specific optimal goal. Numerical simulations have also been performed to review the behaviors of analytical findings. The analytical results have been verified with the numerical simulations. The analysis of this paper shows that it is possible to control drug addiction if there is less interaction among general people with the addicted individuals. Family based care, proper medical treatment, awareness and educational programs can be the most effective ways to reduce the adverse effects of drug addiction in a shortest possible time."

In [30]:
# text_input=df_agg[df_agg['section']=='abstract'].sentence.iloc[2]
#
article = {'title': '', 'abstract': text_input}

abstract = article.get('abstract', '')
sentences = sent_tokenize(abstract)
labels = []

In [31]:
text_input

'The present world is facing a devastating reality as drug abuse prevails in every corner of a society. The progress of a country is obstructed due to the excessive practice of taking drugs by the young generation. Like other countries, Bangladesh is also facing this dreadful situation. The multiple use of drug substances leads an individual to a sorrowful destination and for this reason, the natural behavior of human mind is disrupted. An addicted individual may regain his normal life by proper monitoring and treatment. The objective of this study is to analyze a mathematical model on the dynamics of drug abuse in the perspective of Bangladesh and reduce the harmful consequences with effective control policies using the idea of optimal control theory. The model has been solved analytically introducing a specific optimal goal. Numerical simulations have also been performed to review the behaviors of analytical findings. The analytical results have been verified with the numerical simul

In [33]:
discourse_output = discourse_predictor.predict_json({'abstract': abstract})
labels = discourse_output['labels']
pred = claim_predictor.predict_json({'sentences': sentences})
best_paths = model.crf.viterbi_tags(torch.FloatTensor(pred['logits']).unsqueeze(0), 
                                    torch.LongTensor(pred['mask']).unsqueeze(0))
p_claims = 100 * np.array(best_paths[0][0])
p_claims = list(p_claims)

{'sentences': ['The present world is facing a devastating reality as drug abuse prevails in every corner of a society.', 'The progress of a country is obstructed due to the excessive practice of taking drugs by the young generation.', 'Like other countries, Bangladesh is also facing this dreadful situation.', 'The multiple use of drug substances leads an individual to a sorrowful destination and for this reason, the natural behavior of human mind is disrupted.', 'An addicted individual may regain his normal life by proper monitoring and treatment.', 'The objective of this study is to analyze a mathematical model on the dynamics of drug abuse in the perspective of Bangladesh and reduce the harmful consequences with effective control policies using the idea of optimal control theory.', 'The model has been solved analytically introducing a specific optimal goal.', 'Numerical simulations have also been performed to review the behaviors of analytical findings.', 'The analytical results have

In [34]:
data = {'sents': sentences,
        'scores': p_claims,
        'labels': labels,
        'len': len,
        'enumerate': enumerate,
        'zip': zip}
data.update(article)

In [35]:
data['scores']

[0, 0, 0, 0, 0, 0, 0, 0, 0, 100, 100]

In [332]:
# df=pd.read_csv("../Data/cord_titles_abstracts_conclusions.csv")
df=pd.read_csv("../Data/section_sent_texts_with_drug_mentions_ann_200504.csv").drop('Unnamed: 0',axis=1)

In [333]:
df.head()

,cord_uid,sentence_id,section,sentence,w2vVector,drug_terms_used
0,fjmchbew,fjmchbew032256,title,The Author's Response: Case of the Index Patie...,"[0.00944858, -0.05984148, 0.01565553, -0.02196...","lopinavir,ritonavir,lopinavir,ritonavir"
1,ba6v30jg,ba6v30jg1633000,Clinical features and outcomes of the patients,The clinical features and laboratory test resu...,"[-0.0305434912, -0.0297232307, 0.147603706, 0....","alanine,creatine"
2,ba6v30jg,ba6v30jg1633001,Clinical features and outcomes of the patients,"Fever, cough, and dyspnea were the most common...","[-0.01814028, 0.09488559, 0.08022365, -0.07600...","alanine,creatine"
3,ba6v30jg,ba6v30jg1633002,Clinical features and outcomes of the patients,The white blood cell counts varied among these...,"[-0.02238245, -0.02396372, 0.09787606, -0.0587...","alanine,creatine"
4,ba6v30jg,ba6v30jg1633003,Clinical features and outcomes of the patients,The alanine aminotransferase and serum creatin...,"[0.0692584217, 0.131579965, 0.0442873463, 0.04...","alanine,creatine"


In [334]:
df=df.sort_values(by=['cord_uid','section','sentence_id']).reset_index(drop=True)
df_agg=df.groupby(['cord_uid']).agg({"sentence":list,"sentence_id":list})

df_agg=df_agg.reset_index().rename(columns={"sentence":"sentences"})

df_abs=df_agg

In [335]:
df_abs.shape

(558, 3)

In [336]:
# df_abs['discourse_output'] = df_abs.abstract.apply(lambda x:discourse_predictor.predict_json({'abstract': x}))

# df_abs['labels'] = df_abs['discourse_output'].apply(lambda x:x['labels'])

In [337]:
df_abs['pred'] = df_abs.sentences.apply(lambda x:claim_predictor.predict_json({'sentences': x}))
df_abs['best_paths'] = df_abs.pred.apply(lambda x: model.crf.viterbi_tags(torch.FloatTensor(x['logits']).unsqueeze(0), 
                                    torch.LongTensor(x['mask']).unsqueeze(0)))
df_abs['p_claims']=df_abs['best_paths'].apply(lambda x:100*np.array(x[0][0]))

{'sentences': ['Conclusion-The significance of the study stems mainly from the demonstration that the SARS-CoV-2 macro X domain binds ADP ribose.', 'This is the first step needed to justify screens for potential antivirals that bind in place of ADP ribose.', 'More work needs to be done, however, to understand the antiviral potential of such compounds because the biological role for ADP-ribose binding is still not fully understood.', 'Some work with alpha coronaviruses suggest that ADP-ribose binding by the macro X domain is not needed for viral replication.', '13 However, studies with other (+)RNA viruses suggest that macrodomains are essential for virulence.', '14 This work is also noteworthy because the synthetic codon-optimized plasmid reported here produces up to 100 mg of soluble macro X domain protein per liter of E. coli culture, and this protein retains a high affinity for ADP ribose.', 'The protein could be used for structural studies and screening campaigns.', 'Screening assa

{'sentences': ["The term 'flatten the curve', originating from the Centers for Disease Control and Prevention [28] , has been used widely to describe the impact of social distancing interventions.", 'Here, we used a mathematical model to quantify this term by measuring the short-term number of cases, hospitalization, and deaths averted over the first 100 days under four different social distancing interventions and assuming different levels of compliance in the adult population.', 'When looking at the shortterm impact (here considered to be over the first 100 days) of social distancing interventions started early on in the epidemic, our models suggest that only a social distancing intervention involving all the age groups in the population would considerably decrease the number of cases and delay the epidemic the most.', 'However, with as little as 25% reduction in the contact rates in the adult population (combined with 95% in the older adults), one could reduce the number of hospital

{'sentences': ['At the time of writing this commentary (February 2020), the coronavirus COVID-19 epidemic has already resulted in more fatalities compared with the SARS and MERS coronavirus epidemics combined.', 'Therapeutics that may assist to contain its rapid spread and reduce its high mortality rates are urgently needed.', 'Developing vaccines against the SARS-CoV-2 virus may take many months.', 'Moreover, vaccines based on viral-encoded peptides may not be effective against future coronavirus epidemics, as virus mutations could make them futile.', 'Indeed, new Influenza virus strains emerge every year, requiring new immunizations.', 'A tentative suggestion based on existing therapeutics, which would likely be resistant to new coronavirus mutations, is to use available angiotensin receptor 1 (AT1R) blockers, such as losartan, as therapeutics for reducing the aggressiveness and mortality from SARS-CoV-2 virus infections.', 'This idea is based on observations that the angiotensin-con

{'sentences': ['Role of Chloroquine and Hydroxychloroquine in the Treatment of COVID-19 Infection- A Systematic Literature Review', 'Background: Coronavirus pandemic is currently a global public health emergency.', 'At present, no pharmacological treatment is known to treat this condition, and there is a need to review the available treatments.', 'Objective: While there have been studies to describe the role of chloroquine and hydroxychloroquine in various viral conditions, there is limited information about the use of them in COVID-19.', 'This systematic review aims to summarize the available evidence regarding the role of chloroquine in treating coronavirus infection.', 'The preferred reporting items for systematic reviews and meta-analyses (preferred reporting items for systematic reviews and meta-analyses', 'were pre-prints (not yet peer-reviewed) articles) in this systematic review.', 'All the articles mentioned about the role of chloroquine and /or hydroxychloroquine in limiting 

{'sentences': ['(methylprednisolone 40-120 mg per day) was given as a combined regimen if severe community-acquired pneumonia was diagnosed by physicians at the designated hospital.', 'Oxygen support (eg, nasal cannula and invasive mechanical ventilation) was administered to patients according to the severity of hypoxaemia.', 'Repeated tests for 2019-nCoV were done in patients confirmed to have 2019-nCoV infection to show viral clearance before hospital discharge or discontinuation of isolation.', 'We report here a cohort of 41 patients with laboratoryconfirmed 2019-nCoV infection.', 'Patients had serious, sometimes fatal, pneumonia and were admitted to the designated hospital in Wuhan, China, by Jan 2, 2020.', 'Clinical presentations greatly resemble SARS-CoV. Patients with severe illness developed ARDS and required ICU admission and oxygen therapy.', 'The time between hospital admission and ARDS was as short as 2 days.', 'At this stage, the mortality rate is high for 2019-nCoV, becau

...

{'sentences': ['The pandemic outbreak of COVID-19 is rapidly spreading all over the world, resulting in over one hundred thousand global death due to no any well-established treatment.', 'To our knowledge, present study, a multicenter retrospective cohort study, is the first clinical research on the efficiency of IVIG treatment on critical COVID-19 patients and with the largest number of cases.', 'Patients received IVIG were in more severe condition.', 'However, for the critical COVID-19 patients, the results showed that IVIG has no effect on the 28-day mortality and 60day mortality.', 'Notably, multivariable regression showed that both classification of COVID-19 and IVIG using were the factors which related with hazards ratios of death.', 'Subgroup analysis showed that only in patients with critical type, IVIG could significantly reduce the 28-day mortality rate and prolong the survival time, decrease the inflammatory response and improve some organ functions (all P< 0.05), and IVIG h

{'sentences': ['Background: Recently, Coronavirus Disease 2019 (COVID-19', 'Objectives: To summary and analyze the clinical characteristics and identify the predictors of disease severity and mortality.', 'The PubMed, Web of Science Core Collection, Embase, Cochrane and MedRxiv databases were searched through February 25, 2020.', 'Meta-analysis of Observational Studies in Epidemiology (Meta-analysis of Observational Studies in Epidemiology', 'A total of 30 studies including 53000 patients with were included in this study, the mean age was 49.8 years (95% CI, 47.5-52.2 yrs) and 55.5% were male.', 'The pooled incidence of severity and mortality were 20.2% (95% CI, 15.1-25.2%) and 3.1% (95% CI, 1.9-4.2%), respectively.', 'The predictor for disease severity included old age (≥ 50 yrs, odds ratio [OR] = 2.61; 95% CI, 2.29-2.98), male (OR =1.348, 95% CI, 1.195-1.521), smoking (OR =1.734, 95% CI, All rights reserved.', 'No reuse allowed without permission.', 'author/funder, who has granted me

{'sentences': ["[{'text': 'The infection of a novel coronavirus found in Wuhan of China (SARS-CoV-2) is rapidly spreading, and the incidence rate is increasing worldwide.", 'Due to the lack of effective treatment options for SARS-CoV-2, various strategies are being tested in China, including drug repurposing.', 'In this study, we used our pre-trained deep learning-based drug-target interaction model called Molecule Transformer-Drug Target Interaction (MT-DTI) to identify commercially available drugs that could act on viral proteins of SARS-CoV-2.', 'The result showed that atazanavir, an antiretroviral medication used to treat and prevent the human immunodeficiency virus (HIV), is the best chemical compound, showing an inhibitory potency with Kd of 94.94 nM against the SARS-CoV-2 3C-like proteinase, followed by remdesivir (113.13 nM), efavirenz (199.17 nM), ritonavir (204.05 nM), and dolutegravir (336.91 nM).', 'Interestingly, lopinavir, ritonavir, and darunavir are all designed to targ

{'sentences': ['Objective To explore the clinical characteristics and prognosis of the new coronavirus 2019-nCoV patients combined with cardiovascular disease (CVD).', 'Methods A retrospective analysis was performed on 112 COVID-19 patients with CVD admitted to the western district of Union Hospital in Wuhan, from January 20, 2020 to February 15, 2020.', 'They were divided into critical group (ICU, n =16) and general group ( n =96) according to the severity of the disease and patients were followed up to the clinical endpoint.', 'The observation indicators included total blood count, C-reactive protein (CRP), arterial blood gas analysis, myocardial injury markers, coagulation function, liver and kidney function, electrolyte, procalcitonin (PCT), B-type natriuretic peptide (BNP), blood lipid, pulmonary CT and pathogen detection.', 'Results Compared with the general group, the lymphocyte count (0.74&times;10 9 (0.34&times;10 9 , 0.94&times;10 9 )/L vs. 0.99&times;10 9 (0.71&times;10 9 , 

{'sentences': ['Although recent studies have reported epidemiological and clinical features of COVID-19 [1] , this is the first study to characterize KTx recipients with COVID-19.', 'After the outbreak of COVID-19, we identified five infected KTx recipients.', 'While three patients had a suspected history of exposure, the incubation period concerning the other two patients was unclear (Fig. 1) .', 'However, these two patients were residents of Wuhan, the center of the outbreak, and their infection may have been due to exposure to undiagnosed, asymptomatic carriers of SARS-CoV-2.', 'Furthermore, specifically concerning patient 2, the patient was treated with methylprednisolone pulse therapy, which involves greater immunosuppression and could be a predisposing factor for SARS-CoV-2 infection.', 'Although SARS-CoV-2 nucleic acid reverse transcription polymerase chain reaction analysis using throat swab specimens can confirm COVID-19 infection, false negatives are possible due to the sampl

In [338]:
df_abs['claims']=df_abs.apply(lambda x: np.extract(x['p_claims'],x['sentences']),axis=1)
df_abs['claims_id']=df_abs.apply(lambda x:np.extract(x['p_claims'],x['sentence_id']), axis=1)

In [339]:
df_abs[df_abs['claims'].str.len()>0]['claims'].shape

(345,)

In [340]:
df_abs['claims'].str.len().max()

14

In [341]:
df_abs[df_abs['claims'].str.len()==14]['claims'].iloc[0]

array(['Modeling results indicate that interfering with the salt bridges at hot spot 353 could be an effective strategy for inhibiting binding, and hence for the prevention of coronavirus infections.',
       'We also propose that a glycine residue at the receptor binding domain of the spike glycoprotein can have a critical role in permitting bat variants of the coronaviruses to infect human cells.',
       'These results obtained from independent analysis strongly highlight the importance of 439 to 508 section.',
       'These calculations show that the SARS2, SARS and Rs4231 viruses are predicted to favorably bind to the human hACE2 receptor, while the Rm1 and SARS2-MUT variants present unfavorable free energies of binding.',
       "The fact that the bat's coronavirus Rs4231, in addition to SARS and SARS2, presents favorable interaction with hACE2 is in accordance with the previous observation that it is able to infect human cells expressing this protein (Hu et al., 2017) .",
      

In [342]:
df_claims=df_abs[['cord_uid','claims_id']].rename(columns={"claims_id":"sentence_id"})

In [343]:
lst_col='sentence_id'
df_claims_unstacked=pd.DataFrame({
      col:np.repeat(df_claims[col].values, df_claims[lst_col].str.len())
     for col in df_claims.columns.difference([lst_col])
     }).assign(**{lst_col:np.concatenate(df_claims[lst_col].values)})[df_claims.columns.tolist()]

In [344]:
df_claims_unstacked['claim_flag']=1

In [345]:
df_claims_unstacked.shape

(816, 3)

In [346]:
df.shape

(13872, 6)

In [347]:
df.columns

Index(['cord_uid', 'sentence_id', 'section', 'sentence', 'w2vVector',
       'drug_terms_used'],
      dtype='object')

In [348]:
df_merged=df.merge(df_claims_unstacked,on=['cord_uid','sentence_id'],how="left")
df_merged['claim_flag']=df_merged['claim_flag'].fillna(0)

In [349]:
df_merged.head()

,cord_uid,sentence_id,section,sentence,w2vVector,drug_terms_used,claim_flag
0,02q9y011,02q9y0111022435,RESULTS AND DISCUSSION,Conclusion-The significance of the study stems...,"[0.10180759, -0.00600497, 0.0917831, 0.0254331...","amino acids,glycine,protein c,tandem",0.0
1,02q9y011,02q9y0111022436,RESULTS AND DISCUSSION,This is the first step needed to justify scree...,"[0.00878348, -0.04307965, 0.09269622, -0.06150...","amino acids,glycine,protein c,tandem",0.0
2,02q9y011,02q9y0111022437,RESULTS AND DISCUSSION,"More work needs to be done, however, to unders...","[-0.01625532, 0.02996597, 0.02187862, 0.030338...","amino acids,glycine,protein c,tandem",0.0
3,02q9y011,02q9y0111022438,RESULTS AND DISCUSSION,Some work with alpha coronaviruses suggest tha...,"[0.05328574, 0.00301708, 0.03859519, -0.061556...","amino acids,glycine,protein c,tandem",0.0
4,02q9y011,02q9y0111022439,RESULTS AND DISCUSSION,"13 However, studies with other (+)RNA viruses ...","[0.01838331, -0.0495583, 0.07313509, -0.138604...","amino acids,glycine,protein c,tandem",0.0


In [350]:
df_merged.to_csv("../Output/claims_flag_each_cord_uid.csv",index=False)

In [351]:
df_merged[['cord_uid','claim_flag']].drop_duplicates().shape

(893, 2)

In [353]:
df_merged[df_merged['claim_flag']==1].cord_uid.nunique()

345

In [357]:
key=df_merged.cord_uid.unique()[2]
df_merged[df_merged['cord_uid']==key]['claim_flag'].sum()

2.0

In [358]:
df_merged[df_merged['cord_uid']==key][['sentence','claim_flag']].values

array([['The predominant pathological features of COVID-19 infection largely mimic those previously reported for SARS-CoV-1 infection.',
        0.0],
       ['They include dry cough, persistent fever, progressive dyspnea, and in some cases acute exacerbation of lung function with bilateral pneumonia (30).',
        0.0],
       ['Major lung lesions include several pathological signs, such as diffuse alveolar damage, inflammatory exudation in the alveoli and interstitial tissue, hyperplasia of fibrous tissue, and eventually lung fibrosis (43) (44) (45) .',
        0.0],
       ['It has been shown by fluorescence in situ hybridization technique that SARS-CoV-1 RNA locates to the alveolar pneumocytes and alveolar space (46, 47) .',
        0.0],
       ['Considering all these facts, it is not surprising that most histopathological analyses have been focused on distal parts of the respiratory airways, while the regions other than the alveolus have been less systematically studied.',
     

In [359]:
df_merged[(df_merged['cord_uid']==key)&(df_merged['claim_flag']==1)]['sentence'].values

array(['The results suggest that SARS-CoV infection may target the cell types that are important for the protection of airway mucosa and their damage may lead to deterioration of epithelial cell function, finally leading to a more severe lung disease with accumulation of alveolar exudate and inflammatory cells and lung edema, the signs of pneumonia recently described in the lung specimens of two patients with COVID-19 infection (60) .',
       'The results show putative binding sites for several transcription factors which have a strong correlation of expression with ACE2 in colon, kidney, and ileum.'],
      dtype=object)